In [1]:
import sys
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pprint as pp
from datetime import datetime

In [2]:

# Find repo root (folder containing pyproject.toml)
root = Path.cwd()
while not (root / "pyproject.toml").exists() and root.parent != root:
    root = root.parent

sys.path.insert(0, str(root))  # so `src.*` becomes importable
sys.path.append(os.path.abspath("../../src"))

In [3]:
from src.cross_sell_lift_payment_services.preprocessing import load_raw_data
from src.cross_sell_lift_payment_services.helpers import (eligible_incentives, 
                                                          variants_discounts,
                                                          agg_attach_accepted_paid)

In [4]:
results = load_raw_data("s3://dsml-projects/cross-sell/raw_data_for_model/")

In [37]:
# pandas config options
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

# Data Loading

In [15]:
# Loading data from S3 using the load_raw_data function created in the preprocessing_utils module
df_merchants = results['merchants']
df_cart_events = results['cart_events']
df_cart_incentives = results['cart_incentives']
df_activity_events = results['activity_events']

In [16]:
print("Merchants dataset: \n", df_merchants.shape)
print("-"*100)
print("Cart Events dataset: \n", df_cart_events.shape)
print("-"*100)
print("Cart incentives dataset: \n", df_cart_incentives.shape)
print("-"*100)
print("Activity events dataset: \n", df_activity_events.shape)

Merchants dataset: 
 (6000, 14)
----------------------------------------------------------------------------------------------------
Cart Events dataset: 
 (30000, 13)
----------------------------------------------------------------------------------------------------
Cart incentives dataset: 
 (15582, 21)
----------------------------------------------------------------------------------------------------
Activity events dataset: 
 (120000, 15)


In [17]:
print("Merchants dataset: \n", df_merchants.head(5))
print("-"*100)
print("Cart Events dataset: \n", df_cart_events.head(5))
print("-"*100)
print("Cart incentives dataset: \n", df_cart_incentives.head(5))
print("-"*100)
print("Activity events dataset: \n", df_activity_events.head(5))

Merchants dataset: 
   merchant_id legal_form             mcc  uf sales_channel  \
0     M000000        MEI          beauty  PE        hybrid   
1     M000001        MEI  retail_fashion  BA     in_person   
2     M000002       LTDA  retail_fashion  PR        hybrid   
3     M000003        MEI    food_service  DF     in_person   
4     M000004        MEI     electronics  RS        online   

  preferred_contact_channel onboarding_date  kyc_tier employee_count_band  \
0                     phone      2022-11-09         1                6-10   
1                     email      2023-05-10         1                6-10   
2                     email      2022-11-07         2                 3-5   
3                     email      2024-08-07         3                 1-2   
4                  whatsapp      2023-11-14         1                 1-2   

  city_size_band  has_pix_enabled  has_payment_link_enabled  \
0          large                0                         1   
1          large 

# Preprocessing Steps

In [18]:
df_cart_events["primary_product"] = df_cart_events["products"].apply(lambda s: str(s).split("|")[0] if isinstance(s, str) else None)

In [19]:
df_cart_events ['merchant_cart_id'] = df_cart_events['merchant_id'] + "_" + df_cart_events['cart_id'].astype(str)
df_cart_incentives ['merchant_cart_id'] = df_cart_incentives['merchant_id'] + "_" + df_cart_incentives['cart_id'].astype(str)

In [20]:
merch_carts_incs = df_cart_incentives.merge(df_cart_events[['merchant_cart_id', 
                                                            'primary_product',
                                                            "payment_finalized",
                                                            "cart_value",
                                                            "device_type",
                                                            "payment_method_selected",
                                                            "funnel_step",
                                                            "timestamp" ]], on='merchant_cart_id', how='left', suffixes=("","_cevents"))\
                                     .rename(columns={'timestamp': 'cart_timestamp'})\
                                     .merge(df_merchants, on='merchant_id', how='left', suffixes=("","_merchs"))

In [21]:
print(merch_carts_incs.shape)
merch_carts_incs.head(3)

(15582, 41)


,incentive_id,cart_id,merchant_id,cart_timestamp,placement,incentive_type,candidate_addon,incentive_variant,clicked,accepted,...,sales_channel,preferred_contact_channel,onboarding_date,kyc_tier,employee_count_band,city_size_band,has_pix_enabled,has_payment_link_enabled,has_antecipacao_enabled,has_conta_pj
0,INC0000000,C0008888,M002387,2025-03-03 19:07:00,bundle_box,copy,CONTA_PJ,copy_reminder_conta,0,0,...,hybrid,app_inbox,2024-05-17,2,21+,medium,0,0,0,0
1,INC0000001,C0008888,M002387,2025-03-03 19:10:00,bundle_box,popup,CONTA_PJ,monthly_fee_waived_3m,1,0,...,hybrid,app_inbox,2024-05-17,2,21+,medium,0,0,0,0
2,INC0000002,C0009629,M005416,2025-06-06 08:08:00,payment_moment,copy,PIX_QR,pix_free_first_100,1,0,...,online,app_inbox,2022-11-23,1,1-2,medium,0,0,1,1


In [22]:
merch_carts_incs_exposure = merch_carts_incs
merch_carts_incs_exposure['eligible_incentive'] = merch_carts_incs_exposure.apply(eligible_incentives, axis=1)
merch_carts_incs_exposure['treatment'] = merch_carts_incs_exposure["incentive_variant"].isin(variants_discounts).astype(int)
merch_carts_incs_exposure.head(5)

,incentive_id,cart_id,merchant_id,cart_timestamp,placement,incentive_type,candidate_addon,incentive_variant,clicked,accepted,...,onboarding_date,kyc_tier,employee_count_band,city_size_band,has_pix_enabled,has_payment_link_enabled,has_antecipacao_enabled,has_conta_pj,eligible_incentive,treatment
0,INC0000000,C0008888,M002387,2025-03-03 19:07:00,bundle_box,copy,CONTA_PJ,copy_reminder_conta,0,0,...,2024-05-17,2,21+,medium,0,0,0,0,True,0
1,INC0000001,C0008888,M002387,2025-03-03 19:10:00,bundle_box,popup,CONTA_PJ,monthly_fee_waived_3m,1,0,...,2024-05-17,2,21+,medium,0,0,0,0,True,1
2,INC0000002,C0009629,M005416,2025-06-06 08:08:00,payment_moment,copy,PIX_QR,pix_free_first_100,1,0,...,2022-11-23,1,1-2,medium,0,0,1,1,True,1
3,INC0000003,C0005317,M005205,2025-07-29 06:52:00,bundle_box,copy,ANTECIPACAO,antec_discount_30bps,0,0,...,2022-09-03,3,1-2,small,1,0,0,0,True,1
4,INC0000004,C0007829,M005441,2025-06-19 20:49:00,cart_side_panel,copy,PAYMENT_LINK,link_zero_fee_first_10,0,0,...,2023-08-25,3,11-20,medium,1,1,0,0,False,1


In [23]:
important_cols = {
    "merchants": ["legal_form","mcc","uf","sales_channel","kyc_tier","has_pix_enabled","has_payment_link_enabled","has_antecipacao_enabled","has_conta_pj"],
    "carts": ["products","cart_value","payment_finalized","device_type","payment_method_selected","funnel_step"],
    "incs": ["candidate_addon","incentive_variant","placement","incentive_type","clicked","accepted"]
}

### General Stats

In [24]:
print("Null analysis in the datasets")
nulls = {}
for tbl, cols in important_cols.items():
    df = {"merchants":df_merchants, "carts":df_cart_events, "incs":df_cart_incentives}[tbl]
    nulls[tbl] = {c: float(df[c].isna().mean()) for c in cols if c in df.columns}
printer = pp.PrettyPrinter(width=100, compact=False, sort_dicts=False)
printer.pprint(nulls)

Null analysis in the datasets
{'merchants': {'legal_form': 0.0,
               'mcc': 0.0,
               'uf': 0.0,
               'sales_channel': 0.0,
               'kyc_tier': 0.0,
               'has_pix_enabled': 0.0,
               'has_payment_link_enabled': 0.0,
               'has_antecipacao_enabled': 0.0,
               'has_conta_pj': 0.0},
 'carts': {'products': 0.0,
           'cart_value': 0.0,
           'payment_finalized': 0.0,
           'device_type': 0.0,
           'payment_method_selected': 0.0,
           'funnel_step': 0.0},
 'incs': {'candidate_addon': 0.0,
          'incentive_variant': 0.0,
          'placement': 0.0,
          'incentive_type': 0.0,
          'clicked': 0.0,
          'accepted': 0.0}}


In [26]:
exposed_carts_ratio = round(df_cart_incentives['cart_id'].nunique() / df_cart_events['cart_id'].nunique(), 2)
percentage_incentives_paid =  (merch_carts_incs['payment_finalized'] == 1).mean()
percentage_incentives_paid_events =  (merch_carts_incs['payment_finalized_cevents'] == 1).mean()
percentage_mchts_carts_events = df_merchants['merchant_id'].isin(df_cart_events['merchant_id']).mean()
percentage_mchts_carts_incentives = df_merchants['merchant_id'].isin(df_cart_incentives['merchant_id']).mean()

join_rates = {
    "exposed_cart_ratio": float(exposed_carts_ratio),
    "percentage_incentives_paid": float(percentage_incentives_paid),
    "percentage_incentives_paid_events": float(percentage_incentives_paid_events),
    "pct_merchants_present_in_events": float(percentage_mchts_carts_events),
    "pct_merchants_present_in_incentives": float(percentage_mchts_carts_incentives)
}

printer.pprint(join_rates)

{'exposed_cart_ratio': 0.4,
 'percentage_incentives_paid': 0.7244256193043255,
 'percentage_incentives_paid_events': 0.7244256193043255,
 'pct_merchants_present_in_events': 0.9941666666666666,
 'pct_merchants_present_in_incentives': 0.8675}


# 1. Business Understanding
## 1.1. Insights for product adoption:

#### 1.1.1. Baseline vs Discount performance
- How often merchants accept add-ons when no monetary discount is shown
- How often merchant accept add-ins when momentary discount is shown
- Is the difference between both statistics significant?

In [32]:
# identify if the incentive is a discount
merch_carts_incs['is_discount'] = merch_carts_incs['incentive_variant'].isin(variants_discounts)

In [45]:
payment_acceptances_rate_tbl = merch_carts_incs.groupby(['primary_product',
                                                         'candidate_addon', 
                                                         'eligible_incentive',
                                                         'is_discount'])\
                                               .apply(agg_attach_accepted_paid).reset_index()

C:\Users\castr\AppData\Local\Temp\ipykernel_10564\3963721521.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(agg_attach_accepted_paid).reset_index()


In [46]:
payment_acceptances_rate_tbl['exposure_size'] = payment_acceptances_rate_tbl[0].apply(lambda x:x['exposure_size'])
payment_acceptances_rate_tbl['accept_rate'] = payment_acceptances_rate_tbl[0].apply(lambda x:x['accept_rate'])
payment_acceptances_rate_tbl['paid_acceptance_rate'] = payment_acceptances_rate_tbl[0].apply(lambda x:x['paid_acceptance_rate'])
payment_acceptances_rate_tbl= payment_acceptances_rate_tbl.drop(columns=[0])
payment_acceptances_rate_tbl

,primary_product,candidate_addon,eligible_incentive,is_discount,exposure_size,accept_rate,paid_acceptance_rate
0,Extra_POS,ANTECIPACAO,False,False,60,0.066667,0.066667
1,Extra_POS,ANTECIPACAO,False,True,45,0.111111,0.111111
2,Extra_POS,ANTECIPACAO,True,False,100,0.060000,0.060000
3,Extra_POS,ANTECIPACAO,True,True,81,0.111111,0.111111
4,Extra_POS,CONTA_PJ,True,False,137,0.065693,0.065693
...,...,...,...,...,...,...,...
59,Ton_Super,PAYMENT_LINK,False,True,242,0.086777,0.086777
60,Ton_Super,PAYMENT_LINK,True,False,268,0.041045,0.041045
61,Ton_Super,PAYMENT_LINK,True,True,265,0.075472,0.075472
62,Ton_Super,PIX_QR,True,False,563,0.060391,0.060391


In [48]:
pivot_acceptance_rate_tbl = payment_acceptances_rate_tbl.pivot_table(index=["primary_product","candidate_addon","eligible_incentive"],
                                                                     columns = "is_discount",
                                                                     values = ["exposure_size","accept_rate","paid_acceptance_rate"],
                                                                     fill_value = 0)

In [49]:
pivot_acceptance_rate_tbl

accept_rate           exposure_size        paid_acceptance_rate          
is_discount                                              False     True          False  True                 False     True 
primary_product candidate_addon eligible_incentive                                                                          
Extra_POS       ANTECIPACAO     False                 0.066667  0.111111          60.0   45.0             0.066667  0.111111
                                True                  0.060000  0.111111         100.0   81.0             0.060000  0.111111
                CONTA_PJ        True                  0.065693  0.038095         137.0  105.0             0.065693  0.038095
                PAYMENT_LINK    False                 0.056452  0.078261         124.0  115.0             0.056452  0.078261
                                True                  0.048387  0.045113         124.0  133.0             0.048387  0.045113
                PIX_QR          True                  0.079422  0.126923         277.0  260.0             0.079422  0.126923
Stone_Flex      ANTECIPACAO     False                 0.030769  0.134831          65.0   89.0             0.030769  0.134831
                                True                  0.047904  0.080537         167.0  149.0             0.047904  0.080537
                CONTA_PJ        True                  0.091228  0.123574         570.0  526.0             0.091228  0.123574
                PAYMENT_LINK    False                 0.061224  0.129032         147.0  124.0             0.061224  0.129032
                                True                  0.060403  0.070423         149.0  142.0             0.060403  0.070423
                PIX_QR          True                  0.045845  0.096419         349.0  363.0             0.045845  0.096419
TapTon          ANTECIPACAO     False                 0.045455  0.101124          88.0   89.0             0.045455  0.101124
                                True                  0.040000  0.105634         175.0  142.0             0.040000  0.105634
                CONTA_PJ        True                  0.072340  0.070539         235.0  241.0             0.072340  0.070539
                PAYMENT_LINK    False                 0.112272  0.115044         383.0  339.0             0.112272  0.115044
                                True                  0.094183  0.129973         361.0  377.0             0.094183  0.129973
                PIX_QR          True                  0.049801  0.050209         502.0  478.0             0.049801  0.050209
Ton_Pro         ANTECIPACAO     False                 0.020134  0.071429         149.0  154.0             0.020134  0.071429
                                True                  0.062500  0.067227         256.0  238.0             0.062500  0.067227
                CONTA_PJ        True                  0.026316  0.105263         152.0  171.0             0.026316  0.105263
                EXTRA_POS       True                  0.030568  0.049107         229.0  224.0             0.030568  0.049107
                PAYMENT_LINK    False                 0.050336  0.106250         298.0  320.0             0.050336  0.106250
                                True                  0.079882  0.109422         338.0  329.0             0.079882  0.109422
                PIX_QR          True                  0.091149  0.113182         757.0  751.0             0.091149  0.113182
Ton_Super       ANTECIPACAO     False                 0.044643  0.055046         112.0  109.0             0.044643  0.055046
                                True                  0.043243  0.107143         185.0  196.0             0.043243  0.107143
                CONTA_PJ        True                  0.037594  0.054264         133.0  129.0             0.037594  0.054264
                EXTRA_POS       True                  0.041667  0.093923         168.0  181.0             0.041667  0.093923
                PAYMENT_LINK    False              